In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('peo_info.csv')

In [3]:
df['工龄'].value_counts()

工龄
1    2
5    2
4    2
2    2
6    1
3    1
Name: count, dtype: int64

In [4]:
import pulp
import pandas as pd

# 常数设定
DAYS = 7
LINES = 10
SHIFTS = 3
WORKERS = 42
GROUPS = [8, 8, 4, 8, 8, 6]  # 工人分组
MAX_WORKDAYS = 5
PRODUCTIVITY = [1, 1.1, 1.2, 1.3, 1.4, 1.5]  # 不同工龄的生产力
SHIFT_NAMES = ['早', '午', '晚']
DAY_NAMES = ['Day' + str(d) for d in range(1, DAYS+1)]

# 创建问题实例
model = pulp.LpProblem("Worker_Scheduling", pulp.LpMaximize)

# 工人工龄映射
worker_productivity = {}
cumulative_count = 0
for index, group_size in enumerate(GROUPS):
    for i in range(cumulative_count + 1, cumulative_count + group_size + 1):
        worker_productivity[f'B{str(i).zfill(3)}'] = PRODUCTIVITY[index]
    cumulative_count += group_size

# 决策变量
x = pulp.LpVariable.dicts("x", ((i, j, k, d) for i in range(1, WORKERS+1)
                                for j in range(1, LINES+1)
                                for k in range(1, SHIFTS+1)
                                for d in range(1, DAYS+1)), cat='Binary')
                                
# 目标函数（生产量最大化）
model += pulp.lpSum(worker_productivity[f'B{str(i).zfill(3)}'] * x[(i, j, k, d)]
                    for i in range(1, WORKERS+1)
                    for j in range(1, LINES+1)
                    for k in range(1, SHIFTS+1)
                    for d in range(1, DAYS+1))

# 约束条件
for i in range(1, WORKERS+1):
    model += pulp.lpSum(x[(i, j, k, d)] for j in range(1, LINES+1) 
                        for k in range(1, SHIFTS+1) 
                        for d in range(1, DAYS+1)) <= MAX_WORKDAYS

for i in range(1, WORKERS+1):
    for d in range(1, DAYS+1):
        model += pulp.lpSum(x[(i, j, k, d)] for j in range(1, LINES+1)
                            for k in range(1, SHIFTS+1)) <= 1

for j in range(1, LINES+1):
    for k in range(1, SHIFTS+1):
        for d in range(1, DAYS+1):
            model += pulp.lpSum(x[(i, j, k, d)] for i in range(1, WORKERS+1)) >= 1

# 求解问题
model.solve()

# 收集结果并整理为表格形式
results = {'B' + str(i).zfill(3): [] for i in range(1, WORKERS+1)}
for i in range(1, WORKERS+1):
    for d in range(1, DAYS+1):
        scheduled = False
        for j in range(1, LINES+1):
            for k in range(1, SHIFTS+1):
                if x[(i, j, k, d)].varValue == 1:
                    results[f'B{str(i).zfill(3)}'].append(SHIFT_NAMES[k-1])
                    scheduled = True
        if not scheduled:
            results[f'B{str(i).zfill(3)}'].append('休')

# 创建DataFrame
df = pd.DataFrame(results, index=DAY_NAMES)

# 打印结果
df.to_excel('peo_wk.xlsx')
df

,B001,B002,B003,B004,B005,B006,B007,B008,B009,B010,...,B033,B034,B035,B036,B037,B038,B039,B040,B041,B042
Day1,午,休,午,休,早,早,午,早,早,休,...,午,晚,早,休,早,休,午,晚,休,晚
Day2,午,早,晚,休,早,晚,晚,休,休,休,...,午,休,休,早,休,早,休,午,午,午
Day3,午,早,早,晚,早,早,休,晚,午,晚,...,休,午,休,早,早,早,晚,午,休,晚
Day4,早,晚,休,午,午,休,晚,休,晚,晚,...,休,休,午,早,早,早,早,早,午,休
Day5,休,早,早,早,休,晚,晚,午,早,晚,...,午,午,晚,休,午,晚,午,休,早,休
Day6,休,早,休,早,休,晚,休,午,午,晚,...,午,晚,午,晚,休,早,早,休,晚,晚
Day7,午,休,午,晚,午,休,晚,早,休,早,...,午,早,早,早,早,休,休,早,早,午


In [5]:
import pulp
import pandas as pd

# 常数设定
DAYS = 7
LINES = 10
SHIFTS = 3
WORKERS = 42
GROUPS = [8, 8, 4, 8, 8, 6]  # 工人分组
MAX_WORKDAYS = 5
PRODUCTIVITY = [1, 1.1, 1.2, 1.3, 1.4, 1.5]  # 不同工龄的生产力
SHIFT_NAMES = ['早', '中', '晚']

# 创建问题实例
model = pulp.LpProblem("Worker_Scheduling", pulp.LpMaximize)

# 工人工龄映射
worker_productivity = {}
cumulative_count = 0
for index, group_size in enumerate(GROUPS):
    for i in range(cumulative_count + 1, cumulative_count + group_size + 1):
        worker_productivity[i] = PRODUCTIVITY[index]
    cumulative_count += group_size

# 决策变量
x = pulp.LpVariable.dicts("x", ((i, j, k, d) for i in range(1, WORKERS+1)
                                for j in range(1, LINES+1)
                                for k in range(1, SHIFTS+1)
                                for d in range(1, DAYS+1)), cat='Binary')
                                
# 目标函数（生产量最大化）
model += pulp.lpSum(worker_productivity[i] * x[(i, j, k, d)]
                    for i in range(1, WORKERS+1)
                    for j in range(1, LINES+1)
                    for k in range(1, SHIFTS+1)
                    for d in range(1, DAYS+1))

# 约束条件
for i in range(1, WORKERS+1):
    model += pulp.lpSum(x[(i, j, k, d)] for j in range(1, LINES+1) 
                        for k in range(1, SHIFTS+1) 
                        for d in range(1, DAYS+1)) <= MAX_WORKDAYS

for i in range(1, WORKERS+1):
    for d in range(1, DAYS+1):
        model += pulp.lpSum(x[(i, j, k, d)] for j in range(1, LINES+1)
                            for k in range(1, SHIFTS+1)) <= 1

for j in range(1, LINES+1):
    for k in range(1, SHIFTS+1):
        for d in range(1, DAYS+1):
            model += pulp.lpSum(x[(i, j, k, d)] for i in range(1, WORKERS+1)) >= 1

# 求解问题
model.solve()

# 准备第二个表的数据
shifts_schedule = []

for d in range(1, DAYS+1):
    for k in range(1, SHIFTS+1):
        line_entry = {'日期': d, '班次': SHIFT_NAMES[k-1]}
        for j in range(1, LINES+1):
            line_entry[f'M3{str(j).zfill(2)}'] = ''
            for i in range(1, WORKERS+1):
                if x[(i, j, k, d)].varValue == 1:
                    line_entry[f'M3{str(j).zfill(2)}'] = f'B{str(i).zfill(3)}'
        shifts_schedule.append(line_entry)

# 创建DataFrame
df_shifts = pd.DataFrame(shifts_schedule)
df_shifts.to_excel('M_wk.xlsx')
df_shifts
# 打印结果


,日期,班次,M301,M302,M303,M304,M305,M306,M307,M308,M309,M310
0,1,早,B009,B024,B035,B017,B012,B006,B027,B008,B005,B037
1,1,中,B039,B032,B033,B022,B003,B013,B001,B016,B028,B007
2,1,晚,B034,B030,B042,B021,B018,B023,B031,B011,B019,B040
3,2,早,B038,B036,B032,B027,B002,B026,B018,B029,B025,B005
4,2,中,B041,B031,B024,B028,B033,B019,B040,B001,B013,B042
5,2,晚,B006,B016,B030,B003,B022,B015,B017,B012,B011,B007
6,3,早,B006,B005,B020,B036,B002,B014,B003,B021,B037,B038
7,3,中,B040,B024,B034,B027,B031,B017,B015,B011,B001,B009
8,3,晚,B008,B025,B023,B016,B019,B042,B039,B028,B010,B004
9,4,早,B039,B036,B026,B028,B030,B040,B014,B037,B001,B038
